In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
basedir = '../'
sys.path.append(basedir)

import numpy as np
from numpy.random import RandomState
import pandas as pd
from IPython.display import display

from synth_data import HldaDataGenerator
from hlda.sampler import NCRPNode

Synthetic data test for hierarchical LDA inference.

# 1. Generate Vocab

In [2]:
n_rows = 20
n_cols = 5
vocab_mat = np.zeros((n_rows, n_cols), dtype=np.object)
word_count = 0
for i in range(n_rows):
    for j in range(n_cols):
        vocab_mat[i, j] = 'w%s' % word_count
        word_count += 1
        
print vocab_mat

[['w0' 'w1' 'w2' 'w3' 'w4']
 ['w5' 'w6' 'w7' 'w8' 'w9']
 ['w10' 'w11' 'w12' 'w13' 'w14']
 ['w15' 'w16' 'w17' 'w18' 'w19']
 ['w20' 'w21' 'w22' 'w23' 'w24']
 ['w25' 'w26' 'w27' 'w28' 'w29']
 ['w30' 'w31' 'w32' 'w33' 'w34']
 ['w35' 'w36' 'w37' 'w38' 'w39']
 ['w40' 'w41' 'w42' 'w43' 'w44']
 ['w45' 'w46' 'w47' 'w48' 'w49']
 ['w50' 'w51' 'w52' 'w53' 'w54']
 ['w55' 'w56' 'w57' 'w58' 'w59']
 ['w60' 'w61' 'w62' 'w63' 'w64']
 ['w65' 'w66' 'w67' 'w68' 'w69']
 ['w70' 'w71' 'w72' 'w73' 'w74']
 ['w75' 'w76' 'w77' 'w78' 'w79']
 ['w80' 'w81' 'w82' 'w83' 'w84']
 ['w85' 'w86' 'w87' 'w88' 'w89']
 ['w90' 'w91' 'w92' 'w93' 'w94']
 ['w95' 'w96' 'w97' 'w98' 'w99']]


In [3]:
vocab = vocab_mat.flatten().tolist()
print vocab

['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w18', 'w19', 'w20', 'w21', 'w22', 'w23', 'w24', 'w25', 'w26', 'w27', 'w28', 'w29', 'w30', 'w31', 'w32', 'w33', 'w34', 'w35', 'w36', 'w37', 'w38', 'w39', 'w40', 'w41', 'w42', 'w43', 'w44', 'w45', 'w46', 'w47', 'w48', 'w49', 'w50', 'w51', 'w52', 'w53', 'w54', 'w55', 'w56', 'w57', 'w58', 'w59', 'w60', 'w61', 'w62', 'w63', 'w64', 'w65', 'w66', 'w67', 'w68', 'w69', 'w70', 'w71', 'w72', 'w73', 'w74', 'w75', 'w76', 'w77', 'w78', 'w79', 'w80', 'w81', 'w82', 'w83', 'w84', 'w85', 'w86', 'w87', 'w88', 'w89', 'w90', 'w91', 'w92', 'w93', 'w94', 'w95', 'w96', 'w97', 'w98', 'w99']


# 2. Assign Documents to Tree

In [4]:
NCRPNode.total_nodes = 0
NCRPNode.last_node_id = 0
num_levels = 3
gamma = 1
num_docs = 100

root_node = NCRPNode(num_levels, vocab)
document_path = {}
unique_nodes = set()
unique_nodes.add(root_node)
for d in range(num_docs):

    # populate nodes into the path of this document
    path = np.zeros(num_levels, dtype=np.object)
    path[0] = root_node
    root_node.customers += 1 # always add to the root node first
    for level in range(1, num_levels):
        # at each level, a node is selected by its parent node based on the CRP prior
        parent_node = path[level-1]
        level_node = parent_node.select(gamma)
        level_node.customers += 1
        path[level] = level_node
        unique_nodes.add(level_node)

    # set the leaf node for this document                 
    document_path[d] = path
    
unique_nodes = sorted(unique_nodes, key=lambda x: x.node_id)
print len(unique_nodes)
    
def print_node(node, indent, node_topic):
    out = '    ' * indent
    out += 'node %d (level=%d, documents=%d): ' % (node.node_id, node.level, node.customers)
    if node in node_topic:
        probs, words = node_topic[node]
        out += ' '.join(words)
    print out        
    for child in node.children:
        print_node(child, indent+1, node_topic)        

node_topic = {}
print_node(root_node, 0, node_topic)

11
node 0 (level=0, documents=100): 
    node 1 (level=1, documents=56): 
        node 2 (level=2, documents=23): 
        node 5 (level=2, documents=31): 
        node 10 (level=2, documents=2): 
    node 3 (level=1, documents=35): 
        node 4 (level=2, documents=28): 
        node 6 (level=2, documents=7): 
    node 7 (level=1, documents=9): 
        node 8 (level=2, documents=2): 
        node 9 (level=2, documents=7): 


# 3. Assign Each Node Along the Tree to a Topic

In [5]:
def get_words(vocab_mat, eta, pos, dim):

    if dim == 'row':
        words = vocab_mat[pos]
    elif dim == 'col':
        words = vocab_mat[:, pos]
    
    k = len(words)
    eta = [eta] * k
    probs = np.random.dirichlet(eta)
    return probs, words
    
pos = 0
eta = 1
probs, words = get_words(vocab_mat, eta, pos, 'col')
print probs
print words
print np.sum(probs)

[ 0.00040692  0.03688672  0.09907082  0.02081315  0.0340949   0.06334867
  0.05575553  0.00243414  0.01760897  0.09163744  0.00026633  0.0544931
  0.04593761  0.20827443  0.01494573  0.00392205  0.05150066  0.07604774
  0.02063409  0.10192099]
['w0' 'w5' 'w10' 'w15' 'w20' 'w25' 'w30' 'w35' 'w40' 'w45' 'w50' 'w55'
 'w60' 'w65' 'w70' 'w75' 'w80' 'w85' 'w90' 'w95']
1.0


In [6]:
node_topic = {}
node_topic[unique_nodes[0]] = get_words(vocab_mat, eta, 0, 'row') 
node_topic[unique_nodes[1]] = get_words(vocab_mat, eta, 1, 'row') 
node_topic[unique_nodes[2]] = get_words(vocab_mat, eta, 2, 'row') 
node_topic[unique_nodes[3]] = get_words(vocab_mat, eta, 3, 'row') 
node_topic[unique_nodes[4]] = get_words(vocab_mat, eta, 4, 'row') 
node_topic[unique_nodes[5]] = get_words(vocab_mat, eta, 5, 'row') 
node_topic[unique_nodes[6]] = get_words(vocab_mat, eta, 6, 'row') 
node_topic[unique_nodes[7]] = get_words(vocab_mat, eta, 7, 'row') 
node_topic[unique_nodes[8]] = get_words(vocab_mat, eta, 8, 'row') 
node_topic[unique_nodes[9]] = get_words(vocab_mat, eta, 9, 'row') 
node_topic[unique_nodes[10]] = get_words(vocab_mat, eta, 10, 'row') 
print len(node_topic)

11


In [7]:
print_node(root_node, 0, node_topic)

node 0 (level=0, documents=100): w0 w1 w2 w3 w4
    node 1 (level=1, documents=56): w5 w6 w7 w8 w9
        node 2 (level=2, documents=23): w10 w11 w12 w13 w14
        node 5 (level=2, documents=31): w25 w26 w27 w28 w29
        node 10 (level=2, documents=2): w50 w51 w52 w53 w54
    node 3 (level=1, documents=35): w15 w16 w17 w18 w19
        node 4 (level=2, documents=28): w20 w21 w22 w23 w24
        node 6 (level=2, documents=7): w30 w31 w32 w33 w34
    node 7 (level=1, documents=9): w35 w36 w37 w38 w39
        node 8 (level=2, documents=2): w40 w41 w42 w43 w44
        node 9 (level=2, documents=7): w45 w46 w47 w48 w49


# 4. Generate Words in a Document Based on Its Path

In [8]:
def generate_document(topics, theta, doc_len):

    # for every word in the vocab for this document
    doc = []
    for n in range(doc_len):

        # sample a new topic index    
        k = np.random.multinomial(1, theta).argmax()

        # sample a new word from the word distribution of topic k
        probs, words = topics[k]
        w = np.random.multinomial(1, probs).argmax()
        doc_word = words[w]

        doc.append(doc_word)

    return doc

In [9]:
corpus = []
# alpha = [2.0, 1.0, 0.5]
alpha = [1.0, 1.0, 1.0]
doc_len = 50
for d in range(num_docs):
    path = document_path[d]
    topics = [node_topic[node] for node in path]
    theta = np.random.mtrand.dirichlet(alpha)
    doc = generate_document(topics, theta, doc_len)
    corpus.append(doc)

In [10]:
import os

outdir = '/Users/joewandy/Dropbox/Analysis/hLDA/data/synthetic/'
for d in range(len(corpus)):
    doc = corpus[d]
    file_name = 'doc_%d.txt' % d
    file_path = os.path.join(outdir, file_name)
    with open(file_path, 'w') as f:
        f.write("%s\n" % ' '.join(doc))

# 5. Run hLDA

In [11]:
print len(vocab), len(corpus), len(corpus[0])

100 100 50


convert corpus words into indices

In [12]:
new_corpus = []
for doc in corpus:
    new_doc = []
    for word in doc:
        word_idx = vocab.index(word)
        new_doc.append(word_idx)
    new_corpus.append(new_doc)

In [13]:
print len(vocab), len(new_corpus)
print corpus[0]
print new_corpus[0]

100 100
['w4', 'w13', 'w7', 'w4', 'w9', 'w4', 'w1', 'w4', 'w7', 'w6', 'w14', 'w13', 'w4', 'w4', 'w7', 'w8', 'w4', 'w3', 'w3', 'w2', 'w6', 'w4', 'w8', 'w2', 'w4', 'w14', 'w4', 'w4', 'w2', 'w4', 'w12', 'w4', 'w7', 'w4', 'w3', 'w2', 'w1', 'w10', 'w1', 'w7', 'w2', 'w12', 'w9', 'w13', 'w9', 'w3', 'w6', 'w2', 'w2', 'w10']
[4, 13, 7, 4, 9, 4, 1, 4, 7, 6, 14, 13, 4, 4, 7, 8, 4, 3, 3, 2, 6, 4, 8, 2, 4, 14, 4, 4, 2, 4, 12, 4, 7, 4, 3, 2, 1, 10, 1, 7, 2, 12, 9, 13, 9, 3, 6, 2, 2, 10]


In [14]:
from hlda.sampler import HierarchicalLDA

In [15]:
print alpha, gamma, eta

[1.0, 1.0, 1.0] 1 1


In [19]:
n_samples = 100
hlda = HierarchicalLDA(new_corpus, vocab, alpha=1, gamma=1.0, eta=1.0, num_levels=3)
hlda.estimate(n_samples, display_topics=10, n_words=5, with_weights=False)

HierarchicalLDA sampling

.......... 10
topic=0 level=0 (documents=100): w4, w3, w2, w1, w9, 
    topic=1 level=1 (documents=36): w4, w2, w3, w19, w17, 
        topic=2 level=2 (documents=27): w20, w24, w22, w17, w23, 
        topic=12 level=2 (documents=9): w12, w13, w6, w10, w14, 
    topic=4 level=1 (documents=64): w29, w7, w8, w6, w25, 
        topic=5 level=2 (documents=26): w6, w7, w9, w8, w5, 
        topic=6 level=2 (documents=19): w25, w29, w28, w26, w6, 
        topic=8 level=2 (documents=10): w17, w16, w19, w33, w15, 
        topic=9 level=2 (documents=8): w49, w36, w39, w47, w46, 
        topic=23 level=2 (documents=1): w43, w41, w42, w2, w44, 

.......... 20
topic=0 level=0 (documents=100): w4, w3, w2, w1, w0, 
    topic=1 level=1 (documents=38): w50, w52, w6, w7, w17, 
        topic=2 level=2 (documents=28): w20, w24, w22, w17, w19, 
        topic=12 level=2 (documents=9): w12, w13, w10, w14, w6, 
        topic=33 level=2 (documents=1): w43, w41, w42, w4, w44, 
    topic=